In [72]:
import pyxdf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import os
from sklearn.metrics import mean_squared_error

In [73]:
channels = ['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7'
            , 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz'
            , 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2','AUX_1', 'ACC_X', 'ACC_Y', 'ACC_Z']
removed_channels = ['Fp1', 'F8', 'F7', 'Fp2', 'F3', 'F4']
# TODO ADD AUX WHEN CONNECTED
eye_data_names = ['x', 'y', 'z', 'left_blink', 'right_blink', 'both_blink', 'left_openness', 'right_openness']

columns =  eye_data_names + channels

data, header = pyxdf.load_xdf('data/LSLDataV2/LSLDataLeftFourTimes2/eeg-and-eyetracking.xdf')


In [74]:
def get_time_stamps(stamps: list):
    start = []
    end = []
    more_than_one = False

    for i in range(len(stamps)):
        if i == (len(stamps)-1):
            if(stamps[i-1] == (stamps[i] - 1)):
                end.append(stamps[i])
            break

        if(stamps[i] == (stamps[i+1] - 1)):
            if not more_than_one:
                start.append(stamps[i])
            more_than_one = True
            continue
        else:
            if more_than_one:
                end.append(stamps[i])
                more_than_one = False

    return start, end

In [75]:
def plot_eeg_blink_single_norm(df,df_ts, start, end,column,rang = 50, scale=1, flip = False):
    plt.figure(figsize=(10,6))
    df = df[start - rang:end + rang]
    df.columns = columns
    axis = plt.subplot()
    axis.axvline(df_ts['time'][start], color='green', label = "Blink start - TobiiXR")  # y = 0
    axis.axvline(df_ts['time'][end], color='red', label = "Blink End - TobiiXR")
    df_norm = (df[column].abs() / df[column].abs().max())
    plus = -df_norm[start - rang]
    axis.plot(df_ts['time'][start-rang:end+rang], (df_norm + plus) *(-1 if flip else 1) *scale, label = "EMG")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['left_openness'] / df['left_openness'].abs().max())
               *(-1)+ 1, color='orange', label = "Eye openness - SRanipal")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['right_openness'] / df['right_openness'].abs().max())
               *(-1)+ 1, color='violet', label = "Eye openness right - SRanipal")
    axis.set_xlabel("Time normalized (seconds)")
    axis.set_ylabel("Amplitude ")
    axis.set_title(column)
    plt.legend()
    plt.show()

def plot_eeg_blink_single(df,df_ts, start, end,column,rang = 50):
    plt.figure(figsize=(10,6))
    df = df[start - rang:end + rang]
    df.columns = columns
    axis = plt.subplot()
    axis.axvline(df_ts['time'][start], color='green', label = "Blink start - TobiiXR")  # y = 0
    axis.axvline(df_ts['time'][end], color='red', label = "Blink End - TobiiXR")

    axis.plot(df_ts['time'][start-rang:end+rang], df[column], label = "EMG")
    axis.plot(df_ts['time'][start-rang:end+rang], df['left_openness'], color='orange', label = "Eye openness left - SRanipal")
    axis.plot(df_ts['time'][start-rang:end+rang], df['right_openness'], color='violet', label = "Eye openness right - SRanipal")
    axis.set_title(column)
    plt.legend()
    plt.show()

In [76]:

def init_data(data, shift = 0):
    for s in data:
        stream = s
        y = stream['time_series']
        df_stream = pd.DataFrame(y)
        df_stream_ts = pd.DataFrame(stream['time_stamps'])
        df_stream_ts.columns = ['time']
        df_stream.columns = columns
        df_stream['AUX_1'] = df_stream['AUX_1']+ 1000000
        if shift != 0:
            df_stream['AUX_1'] = df_stream['AUX_1'].shift(shift)
        return stream, df_stream, df_stream_ts

stream, df_stream, df_stream_ts = init_data(data)
def get_blinks_in_df(df):
    left_blinks = df.index[(df['left_blink'] >= 1) & (df['right_blink'] != 1)].tolist()
    right_blinks = df.index[(df['right_blink'] >= 1) & (df['left_blink'] != 1)].tolist()
    both_blink = df.index[df['both_blink'] == 1].tolist()
    return left_blinks,right_blinks, both_blink

left_blinks, right_blinks, both_blink = get_blinks_in_df(df_stream)

In [77]:
avg_time = np.diff(df_stream_ts.to_numpy().flatten()).tolist()

In [78]:
start, end = get_time_stamps(left_blinks)
flip = False
chosen_channel = 'AUX_1'
print(start)
print(end)


[1201, 1510, 1858]
[1278, 1576, 1917]


In [79]:
from scipy.signal import butter, lfilter
# Sample rate and desired cutoff frequencies (in Hz).
def plot_eeg_blink_single_norm(df,df_ts, start, end,column,rang = 50, scale=1, flip = False, order = 4):
    plt.figure(figsize=(10,6))
    df = df[start - rang:end + rang]
    df.columns = columns
    axis = plt.subplot()
    axis.axvline(df_ts['time'][start], color='green', label = "Blink start - TobiiXR")  # y = 0
    axis.axvline(df_ts['time'][end], color='red', label = "Blink End - TobiiXR")
    df_norm = (df[column].abs() / df[column].abs().max())
    plus = -df_norm[start - rang]

    y = (df_norm + plus) *(-1 if flip else 1) *scale
    peaks, _ = signal.find_peaks(y[rang-7:rang+30])
    peaks = [peak + rang-7 for peak in peaks]

    axis.plot(df_ts['time'][start-rang:end+rang], y, label = "EMG")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['left_openness'] / df['left_openness'].abs().max())
               *(-1)+ 1, color='orange', label = "Eye openness - SRanipal")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['right_openness'] / df['right_openness'].abs().max())
               *(-1)+ 1, color='violet', label = "Eye openness right - SRanipal")
    axis.set_xlabel("Time normalized (seconds)")
    axis.set_ylabel("Normalized Amplitude")
    for i in peaks:
        plt.plot(df_ts['time'][i+ start - rang], y[i], marker="o", markersize=5, markeredgecolor="turquoise", markerfacecolor="black", label="Local maximum " + str(y[i].round(2)) + ", " + str(i+ start - rang))
    axis.set_title(column)
    plt.legend()
    plt.show()

def get_peaks(df, start, end, column, rang=50, scale=1, flip=False):
    df = df[start - rang:end + rang]
    df_norm = (df[column].abs() / df[column].abs().max())
    plus = -df_norm[start - rang]
    y = ((df_norm + plus) *(-1 if flip else 1) *scale).tolist()
    peaks, _ = signal.find_peaks(y[rang-20:rang+20])
    peaks = [peak + rang-20 for peak in peaks]
    if len(peaks) == 0:
        #print(df)
        return 0
    peak = peaks[np.argmax([y[i] for i in peaks])]+ start - rang
    return  peak if end + rang > peak > start - rang else start



In [80]:


def find_shift(start, peaks):
    differences_start = []
    differences_end = []
    for i in range(len(peaks)):
        differences_start.append(start[i]-peaks[i])
    return differences_start

def plot_table(df):
    #make this example reproducible

    #define figure and axes
    fig, ax = plt.subplots()
    #hide the axes
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')

    #create data

    #create table
    table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')
    table.scale(2.5, 2.5)
    table.set_fontsize(30)
    plt.show()

In [81]:

def run_files(paths, path='data/LSLDataV3/', both = True, plot = False, shift_init=0):
    peak_info = []
    avg_time = []
    for (index,y) in enumerate(paths):
        data, header = pyxdf.load_xdf(path+str(y)+'/dejittered-fir-eeg-and-dejittered-eyetracking.xdf')
        stream, df_stream, df_stream_ts = init_data(data,shift_init)
        left_blinks, right_blinks, both_blink = get_blinks_in_df(df_stream)
        start, end = get_time_stamps(both_blink if both else left_blinks)
        avg_time.append(np.diff(df_stream_ts.to_numpy().flatten()).tolist())
        peaks = []
        for i in range(len(start)):
            if i == 9:
                continue
            if "LeftTenTimes6" in y or "LeftTenTimes7" in y:
                peaks.append(get_peaks(df_stream, start[i], end[i], chosen_channel, rang=50, scale=5, flip=True))
            else:
                peaks.append(get_peaks(df_stream, start[i], end[i], chosen_channel, rang=50, scale=5, flip=both))

        shift = find_shift(start, peaks)
        peak_info.append(shift)

    n = len(max(peak_info, key=len))
    t = [x + [None]*(n-len(x)) for x in peak_info]
    numpy_array_peak_info = np.array(t)
    df_peak = pd.DataFrame(numpy_array_peak_info)
    df_peak = df_peak.set_index([paths])
    df_peak['time between index (s)'] = [i[0] for i in avg_time]
    df_peak['Sum (indexes)'] = df_peak.sum(axis=1)
    df_peak['Avg (indexes)'] = round(df_peak.mean(axis=1),3)
    df_peak['Avg time (s)'] = round(df_peak['Avg (indexes)']*df_peak['time between index (s)'],5)
    if plot:
        plot_table(df_peak)
    return df_peak



In [82]:
from IPython.core.display_functions import display

paths = [name for name in os.listdir("./data/LSLDataV3")]
both = [name for name in paths if "Both" in name]
left = [name for name in paths if "Left" in name]


df_both = run_files(both, both=True , plot=False)
#df_both = df_both.drop(df_both.columns[[1]], inplace=True, axis=1)
display(df_both)
df_left = run_files(left, both=False , plot=False)
#df_left = df_left.drop(df_left.columns[[1]], inplace=True, axis=1)
display(df_left)


,0,1,2,3,4,5,6,7,8,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLDataBothTenTimes2,-4,-7,-7,-9,-4,-4,-2,-6,-2,0.008,-44.992,-8.180,-0.06544
LSLDataBothTenTimes5,-4,-4,-4,-7,-6,-5,-4,-5,-5,0.008,-43.992,-7.999,-0.06399
LSLDataBothTenTimes4,-4,-5,-6,-6,-4,-5,-7,-7,-6,0.008,-49.992,-9.089,-0.07271
LSLDataBothTenTimes1,-6,-5,-8,-6,-3,-7,-5,-3,-6,0.008,-48.992,-8.908,-0.07126
LSLDataBothTenTimes6,-2,1,-2,-4,-5,-5,-3,-4,-4,0.008,-27.992,-5.089,-0.04071
LSLDataBothTenTimes7,-4,-4,-3,-4,-7,-3,-5,-5,-6,0.008,-40.992,-7.453,-0.05962


,0,1,2,3,4,5,6,7,8,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLDataLeftTenTimes1,-7,-6,-8,-5,-12,-6,-7,-5,-6,0.008,-61.992,-11.271,-0.09017
LSLDataLeftTenTimes7,-4,-4,-3,-5,-4,-1,-4,2,-2,0.008,-24.992,-4.544,-0.03635
LSLDataLeftTenTimes3,-12,-13,-7,-10,-11,-7,-10,-11,-10,0.008,-90.992,-16.544,-0.13235
LSLDataLeftTenTimes4,-12,-16,-9,-13,-8,-18,-4,-17,-14,0.008,-110.992,-20.180,-0.16144
LSLDataLeftTenTimes6,-6,1,-2,1,0,0,-9,1,-5,0.008,-18.992,-3.453,-0.02762
LSLDataLeftTenTimes2,-12,-13,-13,-13,-9,-14,-10,-6,-9,0.008,-98.992,-17.999,-0.14399


In [83]:
def get_offset(df):
    avg_offset = df['Avg time (s)'].sum()/df.shape[0]
    avg_offset_index = df['Avg (indexes)'].sum()/df.shape[0]
    #print(avg_offset_index)
    return avg_offset, avg_offset_index


In [84]:
print("(Seconds, Number of indexes to shift) ---- Both")
print(get_offset(df_both))
print("(Seconds, Number of indexes to shift) ---- Left")
print(get_offset(df_left))

df_both_shift = run_files(both, both=True , plot=False, shift_init=round(get_offset(df_both)[1]))
df_both_shift.loc['Average']= df_both_shift.sum(numeric_only=True, axis=0)/df_both_shift.shape[0]
display(df_both_shift)
df_left_shift = run_files(left, both=False , plot=False, shift_init=round(get_offset(df_left)[1]))
df_left_shift.loc['Average']= df_left_shift.sum(numeric_only=True, axis=0)/df_left_shift.shape[0]
display(df_left_shift)

print(get_offset(df_both_shift))
print(get_offset(df_left_shift))

(Seconds, Number of indexes to shift) ---- Both
(-0.062288333333333334, -7.786333333333334)
(Seconds, Number of indexes to shift) ---- Left
(-0.09865333333333333, -12.331833333333334)


,0,1,2,3,4,5,6,7,8,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLDataBothTenTimes2,4.0,1.0,1.0,-1.0,4.000000,4.000000,6.000000,2.0,6.000000,0.008,27.008000,4.911000,0.039290
LSLDataBothTenTimes5,4.0,4.0,4.0,1.0,2.000000,3.000000,4.000000,3.0,3.000000,0.008,28.008000,5.092000,0.040740
LSLDataBothTenTimes4,4.0,3.0,2.0,2.0,4.000000,3.000000,1.000000,1.0,2.000000,0.008,22.008000,4.001000,0.032010
LSLDataBothTenTimes1,2.0,3.0,0.0,2.0,5.000000,1.000000,3.000000,5.0,2.000000,0.008,23.008000,4.183000,0.033460
LSLDataBothTenTimes6,6.0,9.0,6.0,4.0,3.000000,3.000000,5.000000,4.0,4.000000,0.008,44.008000,8.001000,0.064010
LSLDataBothTenTimes7,4.0,4.0,5.0,4.0,1.000000,5.000000,3.000000,3.0,2.000000,0.008,31.008000,5.638000,0.045100
Average,4.0,4.0,3.0,2.0,3.166667,3.166667,3.666667,3.0,3.166667,0.008,29.174667,5.304333,0.042435


,0,1,2,3,4,5,6,7,8,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLDataLeftTenTimes1,5.0,6.0,4.0,7.0,0.000000,6.000000,5.000000,7.0,6.000000,0.008,46.008,8.365000,0.066920
LSLDataLeftTenTimes7,8.0,8.0,9.0,7.0,8.000000,11.000000,8.000000,14.0,10.000000,0.008,83.008,15.092000,0.120740
LSLDataLeftTenTimes3,0.0,-1.0,5.0,2.0,1.000000,5.000000,2.000000,1.0,2.000000,0.008,17.008,3.092000,0.024740
LSLDataLeftTenTimes4,0.0,-4.0,3.0,-1.0,4.000000,-6.000000,8.000000,-5.0,-2.000000,0.008,-2.992,-0.544000,-0.004350
LSLDataLeftTenTimes6,-16.0,13.0,10.0,13.0,12.000000,12.000000,3.000000,13.0,7.000000,0.008,67.008,12.183000,0.097460
LSLDataLeftTenTimes2,0.0,-1.0,-1.0,-1.0,3.000000,-2.000000,2.000000,6.0,3.000000,0.008,9.008,1.638000,0.013100
Average,-0.5,3.5,5.0,4.5,4.666667,4.333333,4.666667,6.0,4.333333,0.008,36.508,6.637667,0.053102


(0.042435, 5.304333333333333)
(0.053101666666666665, 6.637666666666667)


In [85]:
import statistics
from scipy.stats import sem

standard_de_both = statistics.stdev(df_both['Avg time (s)'])
se_both = sem(df_both['Avg time (s)'])
standard_de_left = statistics.stdev(df_left['Avg time (s)'])
se_left = sem(df_left['Avg time (s)'])

print("Standard devation")
print("Both: " + str(standard_de_both))
print("Left: " + str(standard_de_left))

print("-------- SE ----------")
print("Both: " + str(se_both))
print("Left: " + str(se_left))

Standard devation
Both: 0.011616153264599544
Left: 0.056802202304722896
-------- SE ----------
Both: 0.004742274712038985
Left: 0.023189401985485626


In [86]:
standard_de_both_shift = statistics.stdev(df_both_shift['Avg time (s)'])
se_both_shift = sem(df_both_shift['Avg time (s)'])
standard_de_left_shift = statistics.stdev(df_left_shift['Avg time (s)'])
se_left_shift = sem(df_left_shift['Avg time (s)'])

print("Standard devation - After shift")
print("Both: " + str(standard_de_both_shift))
print("Left: " + str(standard_de_left_shift))

print("-------- SE - After shift ----------")
print("Both: " + str(se_both_shift))
print("Left: " + str(se_left_shift))

Standard devation - After shift
Both: 0.010603286833178977
Left: 0.04553160748925471
-------- SE - After shift ----------
Both: 0.00400766572006817
Left: 0.01720933002993914
